In [ ]:
#First install-> !pip install pesq

import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
from pesq import pesq

def analyze_audio(audio_file):
    """
    Analyzes the audio file for noise level, SNR, and noise spectrum.

    Args:
        audio_file (str): Path to the audio file.

    Returns:
        tuple: A tuple containing RMS, noise floor (dB), SNR (dB),
               noise frequency spectrum, and sample rate.
               Returns None if the file cannot be loaded.
    """
    try:
        y, sr = librosa.load(audio_file)

        rms = np.sqrt(np.mean(y**2))

        noise_floor_percentile = 1
        noise_floor_dB = 20 * np.log10(np.percentile(np.abs(y), noise_floor_percentile))

        snr_dB = 20 * np.log10(rms / (10**(noise_floor_dB / 20)))

        stft = librosa.stft(y)
        magnitude_spectrogram = np.abs(stft)

        return rms, noise_floor_dB, snr_dB, magnitude_spectrogram, sr

    except FileNotFoundError:
        print(f"Error: Audio file not found at {audio_file_path}")
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

audio_file_path = "/content/sample_data/denoised_audio.wav"
results = analyze_audio(audio_file_path)

if results:
    rms, noise_floor_dB, snr_dB, noise_spectrum, sr = results
    print(f"RMS Value: {rms}")
    print(f"Noise Floor (dB): {noise_floor_dB}")
    print(f"SNR (dB): {snr_dB}")
    print("Noise Frequency Spectrum:", noise_spectrum)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(librosa.amplitude_to_db(noise_spectrum, ref=np.max),
                             sr=sr, x_axis='time', y_axis='log')
    plt.colorbar(label='dB')
    plt.title('Noise Frequency Spectrum')
    plt.tight_layout()
    plt.show()

#PESQ Analysis

reference_audio, sr_ref = librosa.load("/content/drive/MyDrive/denoising samples/set 2 - only noisy/bus.wav", sr=None)
degraded_audio, sr_deg = librosa.load("/content/sample_data/denoised_audio.wav", sr=None)

if sr_ref != sr_deg:
    degraded_audio = librosa.resample(degraded_audio, sr_deg, sr_ref)
    sr_deg = sr_ref

score = pesq(sr_ref, reference_audio, degraded_audio, 'wb')

print(f"PESQ Score: {score}")